# Analysing Multidimensional Photoemission Spectroscopy (MPES) data with pyARPES
This example shows how to analyse data in the NeXus format with the [pyARPES](https://github.com/chstan/arpes) python package. You'll find details on how to generate such NeXus files in the [convert](./E1%20Convert%20to%20NeXus.ipynb) or [binning](./E2%20Binning%20of%20WSe2.ipynb) example.

In [ ]:
from arpes.plotting.qt_tool import qt_tool
from arpes.plotting.basic_tools import path_tool
from arpes.io import example_data, load_data
from arpes.endstations.plugin.nexus import NeXusEndstation
import xarray as xr
import numpy as np
from pathlib import Path
import h5py as h5

## Download the data
First we set the file directory and create it if not already present.

In [ ]:
FDIR = "TiTe2"
! mkdir -p '{FDIR}'

Since the provided datafile is comparably large (~200MB) it is not directly provided with the example.
You can download the data for [zero](https://zenodo.org/record/5541490/files/TiTe2_0deg.nxs?download=1) and [60 degree](https://zenodo.org/record/5541490/files/TiTe2_60deg.nxs?download=1) from zenodo. Place the files in the FDIR directory specified in the cell above. Under Linux, macOS and in a NORTH container you can directly use the cell below to download the files with curl.

In [ ]:
! curl -o "{FDIR}/TiTe2_0deg.nxs" "https://zenodo.org/records/5541490/files/TiTe2_0deg.nxs?download=1"
! curl -o "{FDIR}/TiTe2_60deg.nxs" "https://zenodo.org/records/5541490/files/TiTe2_60deg.nxs?download=1"

## Load data into a xarray
Extract the NeXus file into an xArray. In the future, mode advanced loading APIs will be developed, transferring all useful metadata.

In [ ]:
h5_TiTe2_0deg = h5.File(f'{FDIR}/TiTe2_0deg.nxs', 'r' )
meas_TiTe2_0deg = xr.DataArray(
    h5_TiTe2_0deg['entry/data/Photoemission intensity'][:],
    coords={'BE': h5_TiTe2_0deg['entry/data/calculated_Energy'][:],
            'kx': h5_TiTe2_0deg['entry/data/calculated_kx'][:],
            'ky': h5_TiTe2_0deg['entry/data/calculated_ky'][:]},
)

h5_TiTe2_60deg = h5.File(f'{FDIR}/TiTe2_60deg.nxs', 'r' )
meas_TiTe2_60deg = xr.DataArray(
    h5_TiTe2_60deg['entry/data/Photoemission intensity'][:],
    coords={'BE': h5_TiTe2_60deg['entry/data/calculated_Energy'][:],
            'kx': h5_TiTe2_60deg['entry/data/calculated_kx'][:],
            'ky': h5_TiTe2_60deg['entry/data/calculated_ky'][:]},
)

## Visualize 0º data
You can use the "Axes" tab to transpose the volume to obtain the optimal panel arrangement.

In [ ]:
qt_tool(meas_TiTe2_0deg)

## Visualize 60º data

In [ ]:
qt_tool(meas_TiTe2_60deg)

## View the difference between the angles
Noticed the asymmetry in the electron pockts at the Fermi surface? Let's make it shine!
We calculate the difference between the two angles and visualize it.
Unfortunately, there are only linear colorscales available in pyARPES.
The resulting data resembles the main observation in this [paper](https://arxiv.org/pdf/2107.07158.pdf).

In [ ]:
qt_tool(meas_TiTe2_0deg - meas_TiTe2_60deg)

## 4D visualization

We can also test 4D data. The procedure is the same: [download](https://zenodo.org/record/4632481/files/201905_MoTe2.nxs?download=1) the data, convert it to xarray and show it with `qt_tool`.

In [ ]:
! curl -o "{FDIR}/MoTe2_dyn.nxs" "https://zenodo.org/records/4632481/files/201905_MoTe2.nxs?download=1"

In [ ]:
h5_MoTe2_dyn = h5.File(f'{FDIR}/MoTe2_dyn.nxs', 'r')
meas_MoTe2_dyn = xr.DataArray(
    h5_MoTe2_dyn['entry/data/Photoemission intensity'][:],
    coords={'tpp': np.squeeze(h5_MoTe2_dyn['entry/data/calculated_Tpp'][:]),
            'BE': np.squeeze(h5_MoTe2_dyn['entry/data/calculated_Energy'][:]),
            'kx': np.squeeze(h5_MoTe2_dyn['entry/data/calculated_kx'][:]),
            'ky': np.squeeze(h5_MoTe2_dyn['entry/data/calculated_ky'][:])},
)

At the Fermi energy, you will see an electron pocket appearing close to time zero and disappearing immediately after wards. This is the signature of a dynamical Lifshitz transition, a change in the topology of the Fermi surface. To learn more, read https://www.science.org/doi/10.1126/sciadv.abd9275'

In [ ]:
qt_tool(meas_MoTe2_dyn)